In [22]:
import pandas as pd
import re
import unittest
import numpy as np
import sqlite3
import os

In [23]:
# Function to modularize physical model
def modularize_physical_model(housing_data, crime_data):
    # Split the `address` column to get the `city`.
    housing_data['city'] = housing_data['address'].str.split(',').str[-5].str.strip()
    housing_data.drop(columns=["address"], inplace=True)

    # Remove the `3` that come with some of the cities
    crime_data['City'] = crime_data['City'].str.replace('3', '')
    
    # Calculate the mode of total_bedrooms
    mode_total_bedrooms = housing_data['total_bedrooms'].mode()[0]

    # Replace null values with the mode
    housing_data['total_bedrooms'].fillna(mode_total_bedrooms, inplace=True)

    return housing_data, crime_data

In [24]:
# Function to perform data quality check
def data_quality_check(housing_data, crime_data):
    # Integrity constraints on the dataset
    housing_constraints = {
        'total_bedrooms': {
            'type': 'notnull'
        }
    }
    crime_constraints = {
        'City': {
            'type': 'notnull'
        }
    }

    # Unit tests to check data requirements
    def test_data_requirements(data, constraints):
        for column, constraint in constraints.items():
            if constraint['type'] == 'notnull':
                assert data[column].notnull().all(), f"Data in column '{column}' contains null values."

    test_data_requirements(housing_data, housing_constraints)
    test_data_requirements(crime_data, crime_constraints)

    # Count checks to ensure completeness
    housing_count = len(housing_data)
    crime_count = len(crime_data)

    assert housing_count > 0, "Housing data is empty."
    assert crime_count > 0, "Crime data is empty."

    return housing_data, crime_data

In [25]:
# Function to build ETL pipeline and create database
def build_etl_pipeline(housing_data, crime_data, database_name):
    conn = sqlite3.connect(database_name)

    # Create housing table
    housing_data.to_sql('housing', conn, if_exists='replace', index=False)

    # Create crime table
    crime_data.to_sql('crime', conn, if_exists='replace', index=False)

    conn.close()



In [26]:
# Perform modularization, data quality check, and ETL pipeline
def perform_data_pipeline(housing_data_path, crime_data_path, database_name):
    # Read housing data
    housing_data = pd.read_csv(housing_data_path)

    # Read crime data
    crime_data = pd.read_csv(crime_data_path)

    # Modularize physical model
    housing_data, crime_data = modularize_physical_model(housing_data, crime_data)

    # Perform data quality check
    housing_data, crime_data = data_quality_check(housing_data, crime_data)

    # Build ETL pipeline and create database
    build_etl_pipeline(housing_data, crime_data, database_name)

# Define the paths and database name
housing_data_path = "C:\\Users\\Tinotenda Mangarai\\Documents\\Data Science internship\\Omdena Data Engineering\\Project\\new.csv"
crime_data_path = "C:\\Users\\Tinotenda Mangarai\\Documents\\Data Science internship\\Omdena Data Engineering\\Project\\ca_offenses_by_city.csv"
database_name = "housing_crime.db"

# Perform the data pipeline
perform_data_pipeline(housing_data_path, crime_data_path, database_name)

In [30]:
import sqlite3
from pathlib import Path

db_path = Path('C:\\Users\\Tinotenda Mangarai\\Documents\\Data Science internship\\Omdena Data Engineering\\Project\\housing_crime.db')
con = sqlite3.connect(db_path)


In [31]:
# Get the cursor
cursor = con.cursor()

# Get the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

# Close the connection
con.close()



housing
crime


In [33]:
import sqlite3

con = sqlite3.connect(Path('C:\\Users\\Tinotenda Mangarai\\Documents\\Data Science internship\\Omdena Data Engineering\\Project\\housing_crime.db'))

cursor = con.execute("SELECT * FROM crime LIMIT 10;")
rows = cursor.fetchall()
rows

[('Adelanto',
  '33,005',
  '212',
  2,
  '14',
  None,
  '48',
  '148',
  '808',
  '434',
  '254',
  '120',
  '24'),
 ('Agoura Hills',
  '20,970',
  '15',
  0,
  '1',
  None,
  '6',
  '8',
  '310',
  '82',
  '217',
  '11',
  '0'),
 ('Alameda',
  '78,613',
  '148',
  2,
  '7',
  None,
  '61',
  '78',
  '1,819',
  '228',
  '1,245',
  '346',
  '18'),
 ('Albany',
  '19,723',
  '34',
  1,
  '6',
  None,
  '16',
  '11',
  '605',
  '95',
  '447',
  '63',
  '0'),
 ('Alhambra',
  '86,175',
  '168',
  1,
  '13',
  None,
  '74',
  '80',
  '1,929',
  '305',
  '1,413',
  '211',
  '6'),
 ('Aliso Viejo',
  '50,751',
  '32',
  0,
  '5',
  None,
  '3',
  '24',
  '363',
  '53',
  '276',
  '34',
  '1'),
 ('Alturas',
  '2,566',
  '35',
  0,
  '13',
  None,
  '2',
  '20',
  '61',
  '9',
  '48',
  '4',
  '4'),
 ('American Canyon',
  '20,687',
  '58',
  1,
  '7',
  None,
  '21',
  '29',
  '634',
  '97',
  '493',
  '44',
  '2'),
 ('Anaheim',
  '349,471',
  '1,271',
  18,
  '129',
  None,
  '439',
  '685',
  

In [34]:
# Get the cursor
cursor = con.cursor()

# Get the table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the table schemas
for table in tables:
    table_name = table[0]
    print(f"Table: {table_name}")
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    for column in columns:
        print(f"Column: {column[1]}, Type: {column[2]}, Nullable: {'YES' if column[3] == 1 else 'NO'}")

    print()

# Close the connection
con.close()

Table: housing
Column: Unnamed: 0, Type: INTEGER, Nullable: NO
Column: longitude, Type: REAL, Nullable: NO
Column: latitude, Type: REAL, Nullable: NO
Column: housing_median_age, Type: REAL, Nullable: NO
Column: total_rooms, Type: REAL, Nullable: NO
Column: total_bedrooms, Type: REAL, Nullable: NO
Column: population, Type: REAL, Nullable: NO
Column: households, Type: REAL, Nullable: NO
Column: median_income, Type: REAL, Nullable: NO
Column: median_house_value, Type: REAL, Nullable: NO
Column: ocean_proximity, Type: TEXT, Nullable: NO
Column: city, Type: TEXT, Nullable: NO

Table: crime
Column: City, Type: TEXT, Nullable: NO
Column: Population, Type: TEXT, Nullable: NO
Column: Violent crime, Type: TEXT, Nullable: NO
Column: Murder and nonnegligent manslaughter, Type: INTEGER, Nullable: NO
Column: Rape (revised definition), Type: TEXT, Nullable: NO
Column: Rape (legacy definition), Type: REAL, Nullable: NO
Column: Robbery, Type: TEXT, Nullable: NO
Column: Aggravated assault, Type: TEXT, N